In [ ]:
import numpy as np
import pandas as pd
import datetime

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Smart Meter and Weather datasets

we select data between 7th Feb 2020 to 11th Aug 2021 becuase the weather dataset contains farthest data in 7th Feb 2020

In [ ]:
# Import Smart Meter dataframe
ls = ""
file_names = ['Bundit.csv', 'Erdi.csv', 'Female_2.csv', 'Female_4.csv', 'Female_6.csv', 'Female_8.csv', 
             'Male_3.csv', 'Male_4.csv', 'Male_5.csv', 'Male_7.csv', 'Research_health_cmu.csv', 'Research_health_suandok.csv',
             'Subject.csv', 'Tech_service.csv', 'Tech.csv', 'Animal.csv', 'Data_science.csv', 'Dean_education.csv', 'Library.csv', 
              'Mechanical.csv', '30years.csv', 'Econ.csv', 'Acc_ba.csv', 'resource_argo.csv', 'Physics_excellent.csv']
raw = {}
for x in file_names:
    raw["{}".format(x[:-4])] = pd.read_csv(ls + x, usecols=["timeIn", "energyConsumtion"], parse_dates=[0],
                   infer_datetime_format=True, index_col='timeIn').loc['2020-02-07':'2021-08-11',:]



In [ ]:
# Import weather dataframe
weather = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/IS project/1_raw_datasets/weather_new.xlsx')
weather['timestamp'] = pd.to_datetime(weather.timestamp, infer_datetime_format=True)  
weather = weather.sort_values(by='timestamp')

weather_df = pd.DataFrame({"time": pd.date_range(start='2020-02-07',end='2021-08-22', freq='15min')})
weather_df[['pm10', 'pm2.5', 'temp', 'humid']] = np.nan

In [ ]:
weather

In [ ]:
'''def fill_nearest_weather(time):
  less_60_min = abs(weather.timestamp - time) < datetime.timedelta(minutes=60)

  if less_60_min.any():
    return weather[weather.timestamp == min(weather.timestamp[less_60_min], key=lambda x: abs(x - time))][col].values[0]

  else: 
    return np.nan'''

In [ ]:
for col in ['pm10', 'pm2.5', 'temp', 'humid']:
  weather_df[col] = weather_df['time'].progress_apply(lambda x: fill_nearest_weather(x))

In [ ]:
pd.DataFrame({'Missing Values' : weather_df.isna().sum(),
              'Percentage' : (weather_df.isna().sum() * 100) / len(weather_df)})

In [ ]:
weather_df.loc['2020-02-07 13:00:00':, :].isna().sum()

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.heatmap(weather_df.isnull(), cbar=False, cmap="YlGnBu")

# Create the Integrated dataframe
- Creating a dataframe with **rows of 15-mins interval** from date 7th Feb 2020 to 12th Aug 2021 and **columns of every building name and weather dataset column name** (pm2.5, pm10, temp, humid)

- Then put NaN values into every single value in the dataframe

- Because we can find weather data furthest day back in 7th Feb 2020)




In [ ]:
# create dataframe of datetime from 07-Feb-2020 to 06-July-2021
weather_df = pd.DataFrame({"Time": pd.date_range(start='2020-02-07', end='2021-08-22', freq='15min')})

# create cols of smart meter and weather dataset
weather_df[['pm10', 'pm2.5', 'temp', 'humid']] = np.nan
#df_ideal.set_index('Time', inplace=True)

weather_df.head()

# Fill Weather data into Integrated dataframe

To fill data into the integrated dataframe that we created above, I write a function to find the nearset time in weather dataset by putting time form the integrated dataframe

In [ ]:
def fill_nearest_weather(time):
  less_7_min = abs(weather.timestamp - time) < datetime.timedelta(minutes=7, seconds=30)

  if less_7_min.any():
    return weather[weather.timestamp == min(weather.timestamp[less_7_min], key=lambda x: abs(x - time))][col].values[0]

  else: 
    return np.nan

In [ ]:
for col in ['pm10', 'pm2.5', 'temp', 'humid']:
  weather_df[col] = weather_df['Time'].progress_apply(lambda x: fill_nearest_weather(x))

In [ ]:
weather_df.ffill().to_csv('/content/drive/MyDrive/Colab Notebooks/IS project/3_imputed_dataset/impute_weather.csv')

# Fill Smart Meter data into Integrated dataframe

To fill data into the integrated dataframe that we created above, I write a function to find the nearset time in smart meter dataset by putting time form the integrated dataframe

In [ ]:
def fill_nearest_meter(time, data):

  less_7_min = abs(data.timeIn - time) < datetime.timedelta(minutes=7, seconds=30)
  
  if less_7_min.any():
    return data[data.timeIn == min(data.timeIn[less_7_min], key=lambda x: abs(x - time))]['energyConsumtion'].values[0]

  else: 
    return np.nan

In [ ]:
# Import Smart Meter dataframe
ls = "/content/drive/My Drive/Colab Notebooks/IS project/1.1_total_raw_datasets/"
file_names = ['total_accba.csv', 'total_bundit.csv', 'total_Female_1.csv', 'total_Female_2.csv', 'total_Female_3.csv', 'total_Female_4.csv',
              'total_Female_5.csv', 'total_Female_6.csv', 'total_Female_8.csv', 'total_Male_3.csv', 'total_Male_4.csv', 'total_Male_5.csv',
              'total_Male_6.csv', 'total_Male_7.csv', 'total_mass_com.csv', 'total_new_dean.csv', 'total_old_dean.csv', 'total_old_art.csv', 
              'total_rh_cmu.csv', 'total_strategic.csv', 'total_language.csv', 'total_rice.csv', 'total_transport.csv', 'total_old_econ.csv', 
              'total_dean_edu.csv', 'total_main_library.csv', 'total_front_elect.csv', 'total_mechanical_2.csv', 'total_30th_top.csv', 'total_serve_techsci.csv',
              'total_csb2_pailom.csv','total_itsc_front.csv', 'total_subject.csv', 'total_rh_suandok.csv', 'total_physic_excl.csv', 'total_icdi.csv',
              'total_rech_society.csv', 'total_rech_argo.csv', 'total_erdi.csv','total_test_animal.csv', 'total_vet.csv', 'total_food_indus.csv']
raw = {}
for x in file_names:
    df_name = "{}".format(x[:-4])
    raw[df_name] = pd.read_csv(ls + x, usecols=["timeIn", "energyConsumtion"], index_col=0)

In [ ]:
raw['total_accba'].index = pd.to_datetime(raw['total_accba'].index, format="%d/%m/%Y %H:%M")
raw['total_bundit'].index = pd.to_datetime(raw['total_bundit'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_1'].index = pd.to_datetime(raw['total_Female_1'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_2'].index = pd.to_datetime(raw['total_Female_2'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_3'].index = pd.to_datetime(raw['total_Female_3'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_4'].index = pd.to_datetime(raw['total_Female_4'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_5'].index = pd.to_datetime(raw['total_Female_5'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_6'].index = pd.to_datetime(raw['total_Female_6'].index, format="%d/%m/%Y %H:%M")
raw['total_Female_8'].index = pd.to_datetime(raw['total_Female_8'].index, format="%d/%m/%Y %H:%M")
raw['total_Male_3'].index = pd.to_datetime(raw['total_Male_3'].index, format="%d/%m/%Y %H:%M")
raw['total_Male_4'].index = pd.to_datetime(raw['total_Male_4'].index, format="%d/%m/%Y %H:%M")
raw['total_Male_5'].index = pd.to_datetime(raw['total_Male_5'].index, format="%d/%m/%Y %H:%M")
raw['total_Male_6'].index = pd.to_datetime(raw['total_Male_6'].index, format="%d/%m/%Y %H:%M")
raw['total_Male_7'].index = pd.to_datetime(raw['total_Male_7'].index, format="%d/%m/%Y %H:%M")
raw['total_mass_com'].index = pd.to_datetime(raw['total_mass_com'].index, format="%d/%m/%Y %H:%M")
raw['total_new_dean'].index = pd.to_datetime(raw['total_new_dean'].index, format="%d/%m/%Y %H:%M")
raw['total_old_dean'].index = pd.to_datetime(raw['total_old_dean'].index, format="%d/%m/%Y %H:%M")
raw['total_old_art'].index = pd.to_datetime(raw['total_old_art'].index, format="%d/%m/%Y %H:%M")
raw['total_rh_cmu'].index = pd.to_datetime(raw['total_rh_cmu'].index, format="%Y-%m-%d %H:%M:%S")
raw['total_strategic'].index = pd.to_datetime(raw['total_strategic'].index, format="%d/%m/%Y %H:%M")
raw['total_language'].index = pd.to_datetime(raw['total_language'].index, format="%Y-%m-%d %H:%M:%S")
raw['total_rice'].index = pd.to_datetime(raw['total_rice'].index, format="%d/%m/%Y %H:%M")
raw['total_transport'].index = pd.to_datetime(raw['total_transport'].index, format="%d/%m/%Y %H:%M")
raw['total_old_econ'].index = pd.to_datetime(raw['total_old_econ'].index, format="%d/%m/%Y %H:%M")
raw['total_dean_edu'].index = pd.to_datetime(raw['total_dean_edu'].index, format="%Y-%m-%d %H:%M:%S")
raw['total_main_library'].index = pd.to_datetime(raw['total_main_library'].index, format="%Y-%m-%d %H:%M:%S")
raw['total_front_elect'].index = pd.to_datetime(raw['total_front_elect'].index, format="%d/%m/%Y %H:%M")
raw['total_mechanical_2'].index = pd.to_datetime(raw['total_mechanical_2'].index, format="%d/%m/%Y %H:%M")
raw['total_30th_top'].index = pd.to_datetime(raw['total_30th_top'].index, format="%d/%m/%Y %H:%M")
raw['total_serve_techsci'].index = pd.to_datetime(raw['total_serve_techsci'].index, format="%d/%m/%Y %H:%M")
raw['total_csb2_pailom'].index = pd.to_datetime(raw['total_csb2_pailom'].index, format="%d/%m/%Y %H:%M")
raw['total_itsc_front'].index = pd.to_datetime(raw['total_itsc_front'].index, format="%d/%m/%Y %H:%M")
raw['total_subject'].index = pd.to_datetime(raw['total_subject'].index, format="%d/%m/%Y %H:%M")
raw['total_rh_suandok'].index = pd.to_datetime(raw['total_rh_suandok'].index, format="%d/%m/%Y %H:%M")
raw['total_physic_excl'].index = pd.to_datetime(raw['total_physic_excl'].index, format="%Y-%m-%d %H:%M:%S")
raw['total_icdi'].index = pd.to_datetime(raw['total_icdi'].index, format="%d/%m/%Y %H:%M")
raw['total_rech_society'].index = pd.to_datetime(raw['total_rech_society'].index, format="%d/%m/%Y %H:%M")
raw['total_rech_argo'].index = pd.to_datetime(raw['total_rech_argo'].index, format="%d/%m/%Y %H:%M")
raw['total_erdi'].index = pd.to_datetime(raw['total_erdi'].index, format="%d/%m/%Y %H:%M")
raw['total_test_animal'].index = pd.to_datetime(raw['total_test_animal'].index, format="%d/%m/%Y %H:%M")
raw['total_vet'].index = pd.to_datetime(raw['total_vet'].index, format="%d/%m/%Y %H:%M")
raw['total_food_indus'].index = pd.to_datetime(raw['total_food_indus'].index, format="%d/%m/%Y %H:%M")

In [ ]:
# create dataframe of datetime from 07-Feb-2020 to 06-July-2021
df_total = pd.DataFrame({"Time": pd.date_range(start='2020-02-01', end='2021-08-22', freq='15min')})

# create cols of smart meter and weather dataset
#df_integrated[['pm10', 'pm2.5', 'temp', 'humid']] = np.nan
df_total[list(raw.keys())] = np.nan
#df_ideal.set_index('Time', inplace=True)

In [ ]:
for name in list(raw.keys()):
  raw[name].reset_index(inplace=True)
  df_total[name] = df_total['Time'].progress_apply(lambda x: fill_nearest_meter(x, raw[name]))

In [ ]:
'''df_test = pd.DataFrame({"Time": pd.date_range(start='2020-02-01', end='2021-08-22', freq='15min')})
df_test[list(raw.keys())] = np.nan
raw['total_Female_1'].reset_index(inplace=True)
time = df_test.iloc[-200]['Time']
vector = raw['total_Female_1'].values
vector[np.where(min(vector[np.where(abs(vector[:,0] - np.array(time)) < datetime.timedelta(minutes=7, seconds=30))][:,0]))]

#!pip install swifter
import swifter

for name in list(raw.keys()):
  raw[name].reset_index(inplace=True)
  df_test[name] = df_test['Time'].swifter.apply(lambda x: fill_nearest_meter(x, raw[name]))
  
'''


In [ ]:
#df_total.to_csv('/content/drive/MyDrive/Colab Notebooks/IS project/2_integrated_dataset/10_total_dataset.csv')